In [1]:
ini_date = '04/01/23 12:00:00'
end_date = '06/01/23 13:00:00'
target_folder="./time-data/evaluation/"

#[9:50, 11/10/2023] Aurora: setup: 20/12/2022 a las 15:00 a. m. hasta el 23/12/2022 a las 15:00 a. m.
#[9:50, 11/10/2023] Aurora: config: 04/01/2023 a las 12:00 a. m. hasta el 06/01/2023 a las 12:00 a. m.

#ini_date = '20/12/22 15:00:00'
#end_date = '23/12/22 15:00:00'
#target_folder="./time-data/setup/"


time_step=15

binary_sensors=["cubiertos", "fridge", "microwave", "ollas"]

tags=["1","2"]


In [2]:
def repaintXY(x,y):
    return int(x*0.08965-196.41+150), int(-y*0.1078+1077)

def repaintXXYY(x,y):
    return int((x+196.41-150)/0.08965), int((y-1077)/(-0.1078))

print(repaintXY(2280,5135))
print(repaintXY(2860,6730))
print(repaintXXYY(209,351))
#print(repaintXY(5300,6450))
#print(repaintXY(6600,5200))

(157, 523)
(209, 351)
(2848, 6734)


In [3]:
import numpy as np
from pandas import read_csv    

import time

from datetime import datetime

def day_time(ti):
    return int((int)(ti/(60*60*24)))

t0 = (int)(datetime.strptime(ini_date, '%d/%m/%y %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0))

tN = (int)(datetime.strptime(end_date, '%d/%m/%y %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN))



nT=(int)(round((tN-t0-1)/time_step))
print(nT)


def absT(tt):
    return (int)(tt*time_step+t0)

def relT(ti):
    return (int)((ti-t0)/time_step)


04/01/23 12:00:00 Init date is 1672830000 day: 19361
06/01/23 13:00:00 End date is 1673006400 day: 19363
11760


In [4]:
user_location={}
for tag in tags:
    user_location[tag]=[]
    series=read_csv(target_folder+tag+".tsv", sep="\t",parse_dates=True,header=None)
    user_location[tag]=  np.array(series.values[:,[1,2,3,4,5]])  
    print(user_location[tag].shape)
print(user_location["1"])

(11760, 5)
(11760, 5)
[['None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None']
 ...
 ['None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None']]


In [5]:
S=np.zeros((len(binary_sensors), nT))
for i,sensor in enumerate(binary_sensors):
    series=read_csv(target_folder+sensor+".tsv", sep="\t",parse_dates=True,header=None)
    S[i]=  np.array(series.values[:,[1]]).flatten()  
    print(S[i].shape)
print(S.shape)
print(np.where( (S[0]==1)))

(11760,)
(11760,)
(11760,)
(11760,)
(4, 11760)
(array([  212,   217,   337,   338,   375,   376,   382,   467,   468,
         492,   495,  1418,  2217,  2218,  2237,  2244,  2246,  2247,
        2250,  2335,  2336,  2823,  2844,  2845,  4771,  4819,  4869,
        4921,  4922,  5201,  5202,  5226,  5227,  6213,  6218,  6220,
        6243,  6244,  6287,  6288,  6289,  6301,  6336,  6337,  6384,
        6385,  6389,  6391,  6392,  6404,  6405,  6517,  6518,  6519,
        6523,  6525,  6553,  6554,  6555,  6559,  6560,  6574,  6578,
        7337,  7951,  7952,  7963,  7972,  7973,  8195,  8206,  8221,
        8222,  8516,  8682, 10845, 10948, 10949, 11016, 11017, 11045,
       11046, 11093, 11273, 11284, 11285, 11510, 11518, 11545, 11569,
       11589, 11663, 11667, 11685], dtype=int64),)


In [6]:
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier

print("SVM evaluation")

total_y=[]
total_pred=[]

for i,sensor in enumerate(binary_sensors):
    index_activation=np.where( (S[i]==1))[0]
    print("Sensor ",sensor, "#",i, "activation:",index_activation)
    
    y_true={}
    gt=read_csv(target_folder+"gt-labelling-sensors/"+sensor+".tsv", sep="\t",parse_dates=True, na_values=0)
    gt=gt.fillna(0)
    for tx,tt in enumerate(gt["Ix"]):
        
        if(sum(gt.values[tx][2:])==0):
            maxI=0
        else:
            maxI=np.argmax(gt.values[tx][2:])+1
        y_true[tt]=(maxI)
        
        
    X=[]
    Y=[]
    for t_activation in gt["Ix"]:
        t=absT(t_activation)
        #los datos de mi auro label (output) + coordenadas de 1 y 2 (input)
        #print("\t time_index:", t_activation, "LABEL:", y_true[t_activation], "in time", datetime.fromtimestamp(t).strftime("%Y-%m-%d %H:%M:%S"), "user 1:",user_location["1"][t_activation], "user 2:",user_location["2"][t_activation])
        
        user1_coords = [float(coord) if coord != 'None' else 0 for coord in user_location["1"][t_activation][:4]]
        user2_coords = [float(coord) if coord != 'None' else 0 for coord in user_location["2"][t_activation][:4]]
        label = y_true[t_activation]
        
        if label == 'None':
            label = 0

        x1, x2, y1, y2 = user1_coords
        x1_2, x2_2, y1_2, y2_2 = user2_coords
        X.append([x1, x2, y1, y2, x1_2, x2_2, y1_2, y2_2])

        Y.append(int(label)) 

    X = np.array(X)
    Y = np.array(Y)
    
    
    #SVM
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
    clf = svm.SVC()
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    print(classification_report(Y_test, Y_pred))
    

    
    total_y.extend(Y_test)
    total_pred.extend(Y_pred)

print("All sensors!", "SVM")    
print(classification_report(total_y, total_pred))    

SVM evaluation
Sensor  cubiertos # 0 activation: [  212   217   337   338   375   376   382   467   468   492   495  1418
  2217  2218  2237  2244  2246  2247  2250  2335  2336  2823  2844  2845
  4771  4819  4869  4921  4922  5201  5202  5226  5227  6213  6218  6220
  6243  6244  6287  6288  6289  6301  6336  6337  6384  6385  6389  6391
  6392  6404  6405  6517  6518  6519  6523  6525  6553  6554  6555  6559
  6560  6574  6578  7337  7951  7952  7963  7972  7973  8195  8206  8221
  8222  8516  8682 10845 10948 10949 11016 11017 11045 11046 11093 11273
 11284 11285 11510 11518 11545 11569 11589 11663 11667 11685]
              precision    recall  f1-score   support

           1       1.00      0.33      0.50        12
           2       0.67      1.00      0.80        16

    accuracy                           0.71        28
   macro avg       0.83      0.67      0.65        28
weighted avg       0.81      0.71      0.67        28

Sensor  fridge # 1 activation: [  212   217   220  

In [7]:
print("RF evaluation")

total_y=[]
total_pred=[]

for i,sensor in enumerate(binary_sensors):
    index_activation=np.where( (S[i]==1))[0]
    print("Sensor ",sensor, "#",i, "activation:",index_activation)
    
    y_true={}
    gt=read_csv(target_folder+"gt-labelling-sensors/"+sensor+".tsv", sep="\t",parse_dates=True, na_values=0)
    gt=gt.fillna(0)
    for tx,tt in enumerate(gt["Ix"]):
        
        if(sum(gt.values[tx][2:])==0):
            maxI=0
        else:
            maxI=np.argmax(gt.values[tx][2:])+1
        y_true[tt]=(maxI)
        
        
    X=[]
    Y=[]
    for t_activation in gt["Ix"]:
        t=absT(t_activation)
        #los datos de mi auro label (output) + coordenadas de 1 y 2 (input)
        #print("\t time_index:", t_activation, "LABEL:", y_true[t_activation], "in time", datetime.fromtimestamp(t).strftime("%Y-%m-%d %H:%M:%S"), "user 1:",user_location["1"][t_activation], "user 2:",user_location["2"][t_activation])
        
        user1_coords = [float(coord) if coord != 'None' else 0 for coord in user_location["1"][t_activation][:4]]
        user2_coords = [float(coord) if coord != 'None' else 0 for coord in user_location["2"][t_activation][:4]]
        label = y_true[t_activation]
        
        if label == 'None':
            label = 0

        x1, x2, y1, y2 = user1_coords
        x1_2, x2_2, y1_2, y2_2 = user2_coords
        X.append([x1, x2, y1, y2, x1_2, x2_2, y1_2, y2_2])

        Y.append(int(label)) 

    X = np.array(X)
    Y = np.array(Y)
    
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    print(classification_report(Y_test, Y_pred))
    

    
    total_y.extend(Y_test)
    total_pred.extend(Y_pred)

print("All sensors!", "RF")    
print(classification_report(total_y, total_pred))    


    

RF evaluation
Sensor  cubiertos # 0 activation: [  212   217   337   338   375   376   382   467   468   492   495  1418
  2217  2218  2237  2244  2246  2247  2250  2335  2336  2823  2844  2845
  4771  4819  4869  4921  4922  5201  5202  5226  5227  6213  6218  6220
  6243  6244  6287  6288  6289  6301  6336  6337  6384  6385  6389  6391
  6392  6404  6405  6517  6518  6519  6523  6525  6553  6554  6555  6559
  6560  6574  6578  7337  7951  7952  7963  7972  7973  8195  8206  8221
  8222  8516  8682 10845 10948 10949 11016 11017 11045 11046 11093 11273
 11284 11285 11510 11518 11545 11569 11589 11663 11667 11685]
              precision    recall  f1-score   support

           1       0.67      0.86      0.75         7
           2       0.90      0.75      0.82        12

    accuracy                           0.79        19
   macro avg       0.78      0.80      0.78        19
weighted avg       0.81      0.79      0.79        19

Sensor  fridge # 1 activation: [  212   217   220   

C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
